Importamos las librerías necesarias

In [395]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

Abrimos la conexión con MySQL

In [396]:
from getpass import getpass
from mysql.connector import connect, Error
def connect_db(host, user):
    try:
    # Si la conexion se da satisfactoriamente, tendremos un mensaje de éxito.
     conn = connect(
                    host=host,
                    user=user,
                    password=getpass("Enter password: ")
                     )    
     print('SUCCESSFUL\n\n')
     return conn
    except Error as e:
    # En caso de tener un error de conexión, lo mostramos
        print(e)

In [397]:
host = "desafiotravel.cdtvtmfza8iy.eu-west-3.rds.amazonaws.com"
user = 'admin'

conn = connect_db(host, user)

SUCCESSFUL




In [398]:
cursor = conn.cursor()

In [399]:
cursor.execute("USE travelspain")

Hacemos una Consulta para ver la provincia de origen de nuestros usuarios y analizarlos

In [400]:
show_db_query = "SELECT * FROM espana"

with conn.cursor() as cursor:
    cursor.execute(show_db_query)
    ls_espana = cursor.fetchall()

In [401]:
df_espana = pd.DataFrame(ls_espana, columns=('COD_PROVINCIAS','PROVINCIAS'))
df_espana.head()

,COD_PROVINCIAS,PROVINCIAS
0,1,Almería
1,2,Cádiz
2,3,Córdoba
3,4,Granada
4,5,Huelva


In [402]:
show_db_query = "SELECT * FROM input"

with conn.cursor() as cursor:
    cursor.execute(show_db_query)
    ls_input = cursor.fetchall()

In [403]:
df_input = pd.DataFrame(ls_input, columns=('ID_INDEX','COD_PROVINCIAS','ENTORNO', 'ACTIVIDADES'))
df_input

,ID_INDEX,COD_PROVINCIAS,ENTORNO,ACTIVIDADES
0,108,16,Ciudad,Cultura
1,109,18,Ciudad,Ocio Nocturno
2,110,15,Playa,Familiar
3,111,2,Montaña,Gastronomia
4,112,3,Playa,Cultura
...,...,...,...,...
103,211,14,Playa,Ocio Nocturno
104,212,14,Playa,Cultura
105,213,46,Playa,Cultura
106,214,1,Playa,Familiar


In [404]:
df_input["COD_PROVINCIAS"]=df_input["COD_PROVINCIAS"].astype("int64")

In [405]:
df_input_espana=pd.merge(df_input, df_espana,on="COD_PROVINCIAS")
df_input_espana

,ID_INDEX,COD_PROVINCIAS,ENTORNO,ACTIVIDADES,PROVINCIAS
0,108,16,Ciudad,Cultura,Cantabria
1,122,16,Playa,Familiar,Cantabria
2,148,16,Ciudad,Cultura,Cantabria
3,170,16,Playa,Familiar,Cantabria
4,174,16,Campo,Cultura,Cantabria
...,...,...,...,...,...
103,191,51,Ciudad,Cultura,Ceuta
104,193,51,Ciudad,Familiar,Ceuta
105,194,42,Montaña,Familiar,Orense
106,195,25,Playa,Gastronomia,Zamora


In [406]:
colores3=df_input_espana['PROVINCIAS'].unique()
colores3

array(['Cantabria', 'Burgos', 'Santa Cruz de Tenerife', 'Cádiz',
       'Córdoba', 'Las Palmas', 'Teruel', 'Baleares', 'Asturias',
       'Huelva', 'Granada', 'Sevilla', 'Almería', 'Ávila', 'Vizcaya',
       'Málaga', 'Lugo', 'La Rioja', 'Guipúzcoa', 'Zaragoza', 'Navarra',
       'Murcia', 'La Coruña', 'Huesca', 'Jaén', 'Pontevedra', 'Salamanca',
       'Ceuta', 'Orense', 'Zamora', 'Gerona'], dtype=object)

In [407]:
contador_provincias=df_input_espana["PROVINCIAS"].value_counts()
total_prov=contador_provincias.sum()
porcentaje_prov=(contador_provincias/total_prov)*100

In [408]:
df_input_espana3=df_input_espana["PROVINCIAS"].value_counts()

In [409]:
df_input_espana2=df_input_espana.copy()
df_input_espana2['SUMA']=1
df_input_espana2

,ID_INDEX,COD_PROVINCIAS,ENTORNO,ACTIVIDADES,PROVINCIAS,SUMA
0,108,16,Ciudad,Cultura,Cantabria,1
1,122,16,Playa,Familiar,Cantabria,1
2,148,16,Ciudad,Cultura,Cantabria,1
3,170,16,Playa,Familiar,Cantabria,1
4,174,16,Campo,Cultura,Cantabria,1
...,...,...,...,...,...,...
103,191,51,Ciudad,Cultura,Ceuta,1
104,193,51,Ciudad,Familiar,Ceuta,1
105,194,42,Montaña,Familiar,Orense,1
106,195,25,Playa,Gastronomia,Zamora,1


In [410]:
df_input_espana2=df_input_espana2.groupby('PROVINCIAS').sum('SUMA')
df_input_espana2.sort_values(by='SUMA', ascending=False, inplace=True)
df_input_espana2=df_input_espana2[:10]
df_input_espana2.reset_index(inplace=True)
df_input_espana2

,PROVINCIAS,ID_INDEX,COD_PROVINCIAS,SUMA
0,Santa Cruz de Tenerife,1859,180,12
1,Baleares,1516,117,9
2,Las Palmas,1488,126,9
3,Cantabria,1290,128,8
4,Cádiz,894,14,7
5,Asturias,1224,84,7
6,Córdoba,955,18,6
7,Huelva,534,20,4
8,Teruel,664,40,4
9,Sevilla,645,32,4


In [411]:
colores4=df_input_espana2['PROVINCIAS'].unique()
colores4

array(['Santa Cruz de Tenerife', 'Baleares', 'Las Palmas', 'Cantabria',
       'Cádiz', 'Asturias', 'Córdoba', 'Huelva', 'Teruel', 'Sevilla'],
      dtype=object)

In [412]:
total_prov=df_input_espana2['SUMA'].sum()
porcentaje_prov=(df_input_espana2['SUMA']/total_prov)*100
porcentaje_prov

0    17.142857
1    12.857143
2    12.857143
3    11.428571
4    10.000000
5    10.000000
6     8.571429
7     5.714286
8     5.714286
9     5.714286
Name: SUMA, dtype: float64

In [413]:
fig = px.bar(df_input_espana2, y=df_input_espana2['PROVINCIAS'].unique(), x=df_input_espana2['SUMA'], text=porcentaje_prov,title="VISITAS POR LUGAR ORIGEN", color=colores4, barmode="overlay", opacity=1)
fig.update_traces(texttemplate='%{text:0.2f}%', textposition='outside')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.update_layout(barmode='overlay',width=1300, height=800,title_font_size=25, title_font_color='black', title_xanchor='auto')
fig.update_layout(title_x=0.5, font_color='black')
fig.update_layout(font_color='black')
fig.update_layout(font_size=12)
fig.update_layout(xaxis_title="PORCENTAJE VISITAS", yaxis_title="PROVINCIAS", legend_title="LEYENDA")
fig.show()

Hacemos una consulta para analizar los datos por tio de entorno que quieren los usuarios 

In [414]:
colores=df_input['ENTORNO'].unique()
colores

contador_entorno=df_input["ENTORNO"].value_counts()
total=contador_entorno.sum()
porcentaje=(contador_entorno/total)*100

In [415]:
contador_entorno=df_input["ENTORNO"].value_counts()

fig = px.pie(df_input, values=contador_entorno, names=df_input["ENTORNO"].unique(), labels='ENTORNO', title="TOTAL DE VISITAS POR ENTORNO")
fig.update_traces(textposition='inside',textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', width=1100, height=700)
fig.update_layout(title_x=0.45, font_color='black')
fig.update_layout(font_color='black')
fig.update_layout(font_size=16)
fig.show()

Hacemos consulta para analizar los datos de los usuarios por tipo de actividad

In [416]:
contador_actividades=df_input["ACTIVIDADES"].value_counts()
contador_actividades

Familiar         37
Cultura          28
Gastronomia      22
Ocio Nocturno    21
Name: ACTIVIDADES, dtype: int64

In [417]:
contador_actividades=df_input["ACTIVIDADES"].value_counts()
contador_actividades

fig = px.pie(df_input, values=contador_actividades, names=df_input["ACTIVIDADES"].unique(), labels='ACTIVIDADES', title="TOTAL DE VISITAS POR ACTIVIDAD")
fig.update_traces(textposition='inside',textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', width=1100, height=700)
fig.update_layout(title_x=0.45, font_color='black')
fig.update_layout(font_color='black')
fig.update_layout(font_size=16)
fig.show()

Hacemos consulta los destinos recomendados a nuestros usuarios

In [418]:
show_db_query = "SELECT * FROM output"

with conn.cursor() as cursor:
    cursor.execute(show_db_query)
    ls_output = cursor.fetchall()

In [419]:
df_output = pd.DataFrame(ls_output, columns=('ID_INDEX','UBICACION','NOMBRE'))
df_output

,ID_INDEX,UBICACION,NOMBRE
0,119,Andalucía,Sevilla
1,120,Comunidad de Madrid,Madrid
2,121,Andalucía,Costa de la Luz
3,122,Madrid,Sierra de Guadarrama
4,123,Cataluña,Tarragona
...,...,...,...
93,212,Comunidad de Madrid,Madrid
94,213,Alicante,Benidorm
95,214,Cataluña,Tarragona
96,215,Cataluña,Tarragona


In [439]:
df_output_copia=df_output.copy()
df_output_copia['SUMA2']=1
df_output_copia

,ID_INDEX,UBICACION,NOMBRE,SUMA2
0,119,Andalucía,Sevilla,1
1,120,Comunidad de Madrid,Madrid,1
2,121,Andalucía,Costa de la Luz,1
3,122,Madrid,Sierra de Guadarrama,1
4,123,Cataluña,Tarragona,1
...,...,...,...,...
93,212,Comunidad de Madrid,Madrid,1
94,213,Alicante,Benidorm,1
95,214,Cataluña,Tarragona,1
96,215,Cataluña,Tarragona,1


In [440]:
df_output_copia=df_output_copia.groupby('NOMBRE').sum('SUMA2')
df_output_copia.sort_values(by='SUMA2', ascending=False, inplace=True)
df_output_copia=df_output_copia[:10]
df_output_copia.reset_index(inplace=True)
df_output_copia

,NOMBRE,ID_INDEX,SUMA2
0,Sierra de Guadarrama,1732,11
1,Sevilla,1564,10
2,San Sebastián,1349,8
3,Valencia,1234,7
4,Barcelona,1038,6
5,Parque de Atracciones de Barcelona,794,5
6,Tarragona,920,5
7,Sierra de Gredos,881,5
8,Madrid,702,4
9,Costa de la Luz,644,4


In [441]:
colores4=df_output_copia['NOMBRE'].unique()
colores4

array(['Sierra de Guadarrama', 'Sevilla', 'San Sebastián', 'Valencia',
       'Barcelona', 'Parque de Atracciones de Barcelona', 'Tarragona',
       'Sierra de Gredos', 'Madrid', 'Costa de la Luz'], dtype=object)

In [442]:
total_destino=df_output_copia['SUMA2'].sum()
porcentaje_destino=(df_output_copia['SUMA2']/total_destino)*100
porcentaje_destino

0    16.923077
1    15.384615
2    12.307692
3    10.769231
4     9.230769
5     7.692308
6     7.692308
7     7.692308
8     6.153846
9     6.153846
Name: SUMA2, dtype: float64

In [443]:
fig = px.bar(df_output_copia, y=df_output_copia['NOMBRE'].unique(), x=df_output_copia['SUMA2'], text=porcentaje_destino,title="DESTINOS RECOMENDADOS POR ChatGPT3", color=colores4, barmode="overlay", opacity=1)
fig.update_traces(texttemplate='%{text:0.2f}%', textposition='outside')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.update_layout(barmode='overlay',width=1300, height=800,title_font_size=25, title_font_color='black', title_xanchor='auto')
fig.update_layout(title_x=0.5, font_color='black')
fig.update_layout(font_color='black')
fig.update_layout(font_size=12)
fig.update_layout(xaxis_title="PORCENTAJE VISITAS", yaxis_title="PROVINCIAS", legend_title="LEYENDA")
fig.show()

In [444]:
# conn.close()